In [1]:
# Require Pandas version >= 2.2.2
import pandas as pd
assert pd.__version__ >= '2.2.2', 'Please update Pandas to version 2.2.2 or newer'
import pandas as pd, os, pyreadr
import warnings
import time
import numpy as  np
import pandas as pd
import pandas as pd
import visualization as viz
import data_loading as dl
import risk_pricing as rp
import statsmodels.api as sm
import statsmodels.formula.api as smf
import importlib
importlib.reload(rp)
importlib.reload(viz)
importlib.reload(dl)
warnings.filterwarnings("ignore")
source_folder_path = "/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/"
modelname = "embeddings_km10_ipcs" # "dicfullmc10thr10defnob40noa0_8_hdp" # Don't forget to clean again when modelname changes
start_time = time.time()
# print the current working directory (for reference)
print("Current Working Directory:", os.getcwd())
os.chdir(source_folder_path)
print("New Working Directory:", os.getcwd())

Current Working Directory: /Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src
New Working Directory: /Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src


In [2]:
# # Creating additional models
# import matplotlib.pyplot as plt
# embeddings = Embeddings()
# EmbeddingsHKRModel.plot_cosine_similarity(["intellectual property", "patent", "knowledge capital", "research and development", "key talent"])
# Save last plot:
# plt.savefig("cosine_similarity.png")
# base_path = "/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/text/"
# dir_path = os.path.join(base_path)
# os.makedirs(dir_path, exist_ok=True)
# figfolder = os.path.join(dir_path, "")
# # Save to the folder:
# plt.savefig(figfolder + "cosine_similarity_terms.png")
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_knocap", "knowledge capital", to_csv = True)
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_patents", "patents", to_csv = True)
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_intcap", "intangible capital", to_csv = True)

In [3]:
# Loading the data from load_dataframes:


In [4]:
import risk_pricing as rp
import importlib
import embedding_tools as et
import visualization as viz
from embedding_tools import Embeddings
from embedding_tools import EmbeddingsHKRModel
importlib.reload(rp)
importlib.reload(viz)
quantiles = 10
# embeddings = Embeddings()
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_kt", "key talent", to_csv = True)
for quantiles in [10]:
    for modelname in ["embeddings_km10_ipcs"]:
        for pfname in ["pfkki3me3mb"]:
            amazon_nov01_short, cequity_mapper, ff3fw, ff5fw, ff3fm, ff5fm, topic_map_unlabeled, comparison_measures, stoxmo_orig, stoxda_orig, stoxwe_orig = \
                dl.load_dataframes(modelname, start_time, clean_again = False)
            cuts = np.linspace(0, 1, quantiles+1).tolist()
            topic_map = viz.label_topic_map(topic_map_unlabeled, modelname, cuts = cuts)
            print("Running Fama-MacBeth regressions: 5 factors")
            fmb_5, df_betas, eret_we, stoxwe_add = rp.famaMacBethFull(stoxwe_orig, cequity_mapper, topic_map, ff5fw, pfname, kki_cuts = cuts, window_size = 52*2, add_innerkk_pf = True)
            fmb_3, _, _, _ = rp.famaMacBethFull(stoxwe_orig, cequity_mapper, topic_map, ff3fw, pfname, kki_cuts = cuts, window_size = 52*2, add_innerkk_pf = True)
            fmb_1, df_betas = rp.famaMacBeth(eret_we, pfname, formula = "eretw ~ 1 + MktRF + HKR", window_size=52*2)
            print("Running Fama-MacBeth regressions: 1 factor")
            #import visualization as viz
            base_path = "/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/text/"
            dir_path = os.path.join(base_path, f"{modelname}_{quantiles}tiles_{pfname}_HKR_SB")
            os.makedirs(dir_path, exist_ok=True)
            figfolder = os.path.join(dir_path, "")
            generate_plots = True
            ##
            viz.explore_topic_map(topic_map, figfolder, generate_plots)
            viz.explore_fmb([fmb_1, fmb_3, fmb_5], figfolder)
            viz.explore_eret_we(eret_we, figfolder)
            viz.tex_compare_kk_measures(comparison_measures, figfolder)
            viz.plot_returns(stoxwe_add, figfolder)
            # viz.amazon_graph(amazon_nov01_short, figfolder)
            # viz.filecounter(figfolder)
            viz.explore_stoxda(stoxda_orig, cequity_mapper, topic_map, figfolder)
            # eret_we, stoxwe_add = rp.process_stoxwe(stoxwe_orig, cequity_mapper, topic_map, ff3fw, pfname, add_innerkk_pf = True, kki_cuts = cuts)
            # svar_qwe = viz.create_svar()
            # eret_we_agg = viz.preprocess_eret_we(eret_we, svar_qwe)
            # eret_qwe_agg = rp.pseudo_monthly(eret_we_agg) 
            # viz.fig_h1b_vs_smb_kkhml(eret_we_agg, figfolder)
            # viz.tex_HKR_vs_mktrf(eret_qwe_agg, figfolder)
            # viz.tex_summary_statistics(eret_we_agg, figfolder)

Tempo de execução depois de carregar os dados: 11.524718999862671
Running Fama-MacBeth regressions: 5 factors


/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_pricing/risk_pricing.py:398: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_pricing/risk_pricing.py:424: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_

Finished processing stoxwe
New kernel
Finished Fama-MacBeth


/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_pricing/risk_pricing.py:398: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_pricing/risk_pricing.py:424: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_

KeyboardInterrupt: 

In [68]:
indexed_df
indexed_df.index.names = ['pfname', 'yw']

# Calculate mean and standard deviation for each pfname
mean_df = indexed_df.groupby('pfname').mean()
std_df = indexed_df.groupby('pfname').std()

merged_df = mean_df.join(std_df, lsuffix='_mean', rsuffix='_std')

# Keep only the columns related to HKR and eretw
columns_to_keep = [col for col in merged_df.columns if 'HKR' in col or 'eretw' in col]
filtered_df = merged_df[columns_to_keep]
filtered_df
filtered_df.index.names = ['pfname']

# Convert the index to a DataFrame to manipulate it
index_df = filtered_df.index.to_frame(index=False)

# Step 1: Add leading zeros to pfname
index_df['pfname'] = index_df['pfname'].astype(str).str.zfill(3)

# Step 2: Create long_pfname
def create_long_pfname(pfname):
    if len(pfname) != 3:
        return pfname
    first_digit, second_digit, third_digit = pfname[0], pfname[1], pfname[2]
    first_char = first_digit
    second_char = 'S' if second_digit == '1' else ('M' if second_digit in '2' else 'B')
    third_char = 'H' if third_digit == '1' else ('M' if third_digit == '2' else ('L' if third_digit == '3' else ''))
    return f"{second_char}/{third_char}/{first_char}"

index_df['long_pfname'] = index_df['pfname'].map(create_long_pfname)

filtered_index_df = index_df[index_df['pfname'].str[0].isin(['0', '5', '9'])]

filtered_index_df = filtered_index_df.set_index('pfname')
if isinstance(filtered_index_df, pd.Index):
    filtered_index_df = filtered_index_df.to_frame(index=False)
filtered_index_df = filtered_index_df.reset_index()
filtered_df = filtered_df.reset_index()
# filtered_df
# merged_df = pd.merge(filtered_df, filtered_index_df, on='pfname')
filtered_df['pfname'] = filtered_df['pfname'].astype(str).str.lstrip('0')
filtered_index_df['pfname'] = filtered_index_df['pfname'].astype(str).str.lstrip('0')
merged_df = pd.merge(filtered_df, filtered_index_df, on='pfname')
merged_df = merged_df.set_index('long_pfname')
merged_df = merged_df.drop(columns=['pfname'])



# Set the modified index back to the DataFrame
#filtered_df.index = filtered_index_df['long_pfname']


# Display the final result
# filtered_df.index = filtered_index['long_pfname']

ValueError: User-provided `key` function must not change the shape of the array.